In [124]:
import textgrid as tg
import os
import numpy as np
import pandas as pd

from pydub import AudioSegment

import h5py

import math
import datetime

from python_speech_features import mfcc
from python_speech_features import delta
from python_speech_features import logfbank

In [140]:
class FeatureExtractor:
    def __init__(self, features):
        self.features = features
        self.df = pd.DataFrame(columns=["label", "features"])
    
    def __loadWav(self, path):
        sig = AudioSegment.from_file(path, format="wav")

        return sig
    
    def __extractFeatures(self,segment):
        samples = np.array(segment.get_array_of_samples())
        frame_rate = segment.frame_rate

        mfcc_feat = mfcc(samples, frame_rate, nfft=1103)
        d_mfcc_feat = delta(mfcc_feat, 2)
        fbank_feat = logfbank(samples, frame_rate, nfft=1103)

        return np.concatenate((np.array(mfcc_feat),np.array(d_mfcc_feat),np.array(fbank_feat)), axis=1)

    def extract(self, path):
        for i, file in enumerate(os.listdir(path)):        
            if file.endswith(".TextGrid"): 
                wav_path = os.path.join(path, os.path.splitext(file)[0] + '.wav')
                if os.path.isfile(wav_path): 
                    sig = self.__loadWav(wav_path)
                    
                    textgrid_path = os.path.join(path, file)
                    textgrid = tg.TextGrid.fromFile(textgrid_path)

                    humorous_tier = textgrid[1]
                    for interval in humorous_tier: #humorous tier
                        if interval.mark in self.features:
                            start = int(interval.minTime * 1000)
                            end = int(interval.maxTime * 1000)
                            trimmed_sig = sig[start:end]
                            label = interval.mark
                            
                            feat = self.__extractFeatures(trimmed_sig).astype(np.float32).tolist()
                            self.df.loc[self.df.shape[0]] = [label,feat] 
        return self.df
    
    def saveFeaturesHDF(self, path):
        filename = 'features{}.h5'.format(datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S"))
        self.df.to_hdf(os.path.join(path, filename),'df',mode='w',format='fixed',data_columns=True, compression='zlib')
    
    def saveFeaturesCSV(self, path):
        filename = 'features{}.csv'.format(datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S"))
        self.df.to_csv(os.path.join(path, filename),mode='w')

In [ ]:
extractor = FeatureExtractor(['H', 'N'])

extractor.extract('data/')

In [139]:
extractor.saveFeaturesCSV('data/')